In [ ]:
from flask import Flask, request, jsonify
from flask_socketio import SocketIO
import psycopg2
import requests
import time

app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="*")

# PostgreSQL connection
conn = psycopg2.connect("dbname=weather_db user=postgres password=admin host=localhost")
cur = conn.cursor()

# OpenWeather API details
API_KEY = "2ffa196ef19c37ec6c9970f6e835ce92"
CITY = "Shah Alam, MY"

# Fetch live weather data
def fetch_weather():
    url = f"http://api.openweathermap.org/data/2.5/weather?q={CITY}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return {
            "city": CITY,
            "temperature": data["main"]["temp"],
            "humidity": data["main"]["humidity"],
            "description": data["weather"][0]["description"]
        }
    return None

# Add weather data
@app.route('/add', methods=['POST'])
def add_weather():
    weather = fetch_weather()
    if weather:
        cur.execute("INSERT INTO weather (city, temperature, humidity, description) VALUES (%s, %s, %s, %s) RETURNING id;",
                    (weather["city"], weather["temperature"], weather["humidity"], weather["description"]))
        conn.commit()
        weather_id = cur.fetchone()[0]
        socketio.emit('weather_update', {"id": weather_id, **weather})
        return jsonify({"message": "Weather data added", "id": weather_id})
    return jsonify({"error": "Failed to fetch weather data"}), 500

# Delete weather data
@app.route('/delete/<int:weather_id>', methods=['DELETE'])
def delete_weather(weather_id):
    cur.execute("DELETE FROM weather WHERE id = %s RETURNING id;", (weather_id,))
    conn.commit()
    socketio.emit('weather_delete', {"id": weather_id})
    return jsonify({"message": "Weather data deleted"})

# Periodically fetch and update weather data
def update_weather_periodically():
    while True:
        weather = fetch_weather()
        if weather:
            cur.execute("INSERT INTO weather (city, temperature, humidity, description) VALUES (%s, %s, %s, %s) RETURNING id;",
                        (weather["city"], weather["temperature"], weather["humidity"], weather["description"]))
            conn.commit()
            weather_id = cur.fetchone()[0]
            socketio.emit('weather_update', {"id": weather_id, **weather})
        time.sleep(60)  # Update every minute

if __name__ == '__main__':
    socketio.start_background_task(update_weather_periodically)
    socketio.run(app, debug=True)
